In [31]:
# Solving and simulating a RBC model using Sims Method
# Writed in Julia 1.5.3 by Wenddy Xu, Simon Fraiser University/AHU/CIMERS/国民经济工程实验室(北京)
# this code is part of chapter 3, "Solving and Simulating DSGE Models" from the book: "Introduction to Quantitative Macroeconomics using Julia"
# Academic Press - Elsevier
# for comments, email at: xuweny87(at)hotmail(dot)com

using LinearAlgebra

# Specify Parameters
alpha = 0.4;
delta = 0.025;
rho  = 0.95;
beta= 0.988;
hs   = 0.5;
sz   = 0.00217;

# Deterministic Steady state

ysk = (1-beta*(1-delta))/(alpha*beta);
ksy = 1/ysk;
si  = delta/ysk;
sc  = 1-si;

# matrices of State Space System
# Define equation indices
eq_ls=1              #labor supply
eq_pf=2              #production function
eq_rc=3              #resource constraints
eq_ec=4              #evolve of capital
eq_ee=5              #eule equation
eq_sh=6              #shock
eq_Ey=7              #error for output
eq_Ec=8              #error for consumption

# variable indices
v_y=1
v_c=2
v_h=3
v_i=4
v_k=5
v_a=6
v_Ey=7
v_Ec=8

# shock indices
e_a=1

# expectation error indices
n_y=1
n_c=2

# system summary
neq=8       # number of equations
neps=1      # number of shocks
neta=2      #number of expectation errors

# initialize matrices
TAU0=zeros(neq,neq)      # $\Tau_0$
TAU1=zeros(neq,neq)      # $\Tau_1$
PHI=zeros(neq,neps)      # $\Phi$
PI=zeros(neq,neta)       # $\Pi$

# produce the state space solution of the RBC given by:
# TAU0*y(t)=TAU1*y(t-1)+PHI*eps(t)+PI*eta(t)

# 1. labor supply
TAU0[eq_ls,v_y]=-1
TAU0[eq_ls,v_c]=1
TAU0[eq_ls,v_h]=1+hs

# 2. production function
TAU0[eq_pf,v_y]=1
TAU0[eq_pf,v_h]=alpha-1
TAU0[eq_pf,v_k]=-alpha
TAU0[eq_pf,v_a]=-1

# 3. resource constraints
TAU0[eq_rc,v_y]=1
TAU0[eq_rc,v_c]=-sc
TAU0[eq_rc,v_i]=-si

# 4. evolve of capital
TAU0[eq_ec,v_k]=1
TAU0[eq_ec,v_i]=-1
TAU1[eq_ec,v_k]=delta-1

# 5. Eule equation
TAU0[eq_rc,v_c]=-1
TAU0[eq_rc,v_k]=1-beta*(1-delta)
TAU0[eq_rc,v_Ey]=-(1-beta*(1-delta))
TAU0[eq_rc,v_Ec]=1

# 6. shock
TAU0[eq_sh,v_a]=1
TAU1[eq_sh,v_a]=rho
PHI[eq_sh,e_a]=1

# Expectation Error
# 7. Ey
TAU0[eq_Ey,v_y]=1
TAU1[eq_Ey,v_Ey]=1
PI[eq_Ey,n_y]=1

# 8. Ec
TAU0[eq_Ec,v_c]=1
TAU1[eq_Ec,v_Ec]=1
PI[eq_Ec,n_c]=1



1

In [33]:
TAU0

8×8 Array{Float64,2}:
 -1.0   1.0   1.5   0.0       0.0      0.0   0.0     0.0
  1.0   0.0  -0.6   0.0      -0.4     -1.0   0.0     0.0
  1.0  -1.0   0.0  -0.26921   0.0367   0.0  -0.0367  1.0
  0.0   0.0   0.0  -1.0       1.0      0.0   0.0     0.0
  0.0   0.0   0.0   0.0       0.0      0.0   0.0     0.0
  0.0   0.0   0.0   0.0       0.0      1.0   0.0     0.0
  1.0   0.0   0.0   0.0       0.0      0.0   0.0     0.0
  0.0   1.0   0.0   0.0       0.0      0.0   0.0     0.0

In [34]:
TAU1

8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0   0.0    0.0   0.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.0   0.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.0   0.0  0.0
 0.0  0.0  0.0  0.0  -0.975  0.0   0.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.0   0.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.95  0.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.0   1.0  0.0
 0.0  0.0  0.0  0.0   0.0    0.0   0.0  1.0

In [26]:
PHI

8×1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0

In [27]:
PI

8×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 1.0  0.0
 0.0  1.0

In [35]:
r = schur(complex(TAU0),complex(TAU1))

GeneralizedSchur{Complex{Float64},Array{Complex{Float64},2}}
S factor:
8×8 Array{Complex{Float64},2}:
 0.449117-0.0im  -0.319497+0.0im  …      0.0+0.0im   0.0+0.0im
      0.0+0.0im   -1.18183+0.0im         0.0+0.0im   0.0+0.0im
      0.0+0.0im        0.0+0.0im     -0.0367+0.0im   0.0+0.0im
      0.0+0.0im        0.0+0.0im         0.0+0.0im   0.0+0.0im
      0.0+0.0im        0.0+0.0im         0.0+0.0im  -1.0+0.0im
      0.0+0.0im        0.0+0.0im  …      0.0+0.0im   0.0+0.0im
      0.0+0.0im        0.0+0.0im         0.0+0.0im   0.0+0.0im
      0.0+0.0im        0.0+0.0im         0.0+0.0im   1.0+0.0im
T factor:
8×8 Array{Complex{Float64},2}:
 0.636696+0.0im  0.381183+0.0im  …  0.0+0.0im  0.0+0.0im   0.0+0.0im
      0.0+0.0im  0.218815+0.0im     0.0+0.0im  0.0+0.0im   0.0+0.0im
      0.0+0.0im       0.0+0.0im     0.0+0.0im  0.0+0.0im   0.0+0.0im
      0.0+0.0im       0.0+0.0im     0.0+0.0im  0.0+0.0im   0.0+0.0im
      0.0+0.0im       0.0+0.0im     0.0+0.0im  0.0+0.0im   0.0+0.0im
      0.

In [38]:
sum(abs.(diag(r.T)./diag(r.S)).<1)

6

In [20]:
?schur

search: schur schur! Schur ordschur ordschur! GeneralizedSchur schedule



```
schur(A::StridedMatrix) -> F::Schur
```

Computes the Schur factorization of the matrix `A`. The (quasi) triangular Schur factor can be obtained from the `Schur` object `F` with either `F.Schur` or `F.T` and the orthogonal/unitary Schur vectors can be obtained with `F.vectors` or `F.Z` such that `A = F.vectors * F.Schur * F.vectors'`. The eigenvalues of `A` can be obtained with `F.values`.

Iterating the decomposition produces the components `F.T`, `F.Z`, and `F.values`.

# Examples

```jldoctest
julia> A = [5. 7.; -2. -4.]
2×2 Array{Float64,2}:
  5.0   7.0
 -2.0  -4.0

julia> F = schur(A)
Schur{Float64,Array{Float64,2}}
T factor:
2×2 Array{Float64,2}:
 3.0   9.0
 0.0  -2.0
Z factor:
2×2 Array{Float64,2}:
  0.961524  0.274721
 -0.274721  0.961524
eigenvalues:
2-element Array{Float64,1}:
  3.0
 -2.0

julia> F.vectors * F.Schur * F.vectors'
2×2 Array{Float64,2}:
  5.0   7.0
 -2.0  -4.0

julia> t, z, vals = F; # destructuring via iteration

julia> t == F.T && z == F.Z && vals == F.values
true
```

---

```
schur(A::StridedMatrix, B::StridedMatrix) -> F::GeneralizedSchur
```

Computes the Generalized Schur (or QZ) factorization of the matrices `A` and `B`. The (quasi) triangular Schur factors can be obtained from the `Schur` object `F` with `F.S` and `F.T`, the left unitary/orthogonal Schur vectors can be obtained with `F.left` or `F.Q` and the right unitary/orthogonal Schur vectors can be obtained with `F.right` or `F.Z` such that `A=F.left*F.S*F.right'` and `B=F.left*F.T*F.right'`. The generalized eigenvalues of `A` and `B` can be obtained with `F.α./F.β`.

Iterating the decomposition produces the components `F.S`, `F.T`, `F.Q`, `F.Z`, `F.α`, and `F.β`.


In [2]:
?svd

search: svd svd! SVD svdvals svdvals! GeneralizedSVD isvalid CholeskyPivoted



```
svd(A; full::Bool = false, alg::Algorithm = default_svd_alg(A)) -> SVD
```

Compute the singular value decomposition (SVD) of `A` and return an `SVD` object.

`U`, `S`, `V` and `Vt` can be obtained from the factorization `F` with `F.U`, `F.S`, `F.V` and `F.Vt`, such that `A = U * Diagonal(S) * Vt`. The algorithm produces `Vt` and hence `Vt` is more efficient to extract than `V`. The singular values in `S` are sorted in descending order.

Iterating the decomposition produces the components `U`, `S`, and `V`.

If `full = false` (default), a "thin" SVD is returned. For a $M \times N$ matrix `A`, in the full factorization `U` is `M \times M` and `V` is `N \times N`, while in the thin factorization `U` is `M \times K` and `V` is `N \times K`, where `K = \min(M,N)` is the number of singular values.

If `alg = DivideAndConquer()` a divide-and-conquer algorithm is used to calculate the SVD. Another (typically slower but more accurate) option is `alg = QRIteration()`.

!!! compat "Julia 1.3"
    The `alg` keyword argument requires Julia 1.3 or later.


# Examples

```jldoctest
julia> A = rand(4,3);

julia> F = svd(A); # Store the Factorization Object

julia> A ≈ F.U * Diagonal(F.S) * F.Vt
true

julia> U, S, V = F; # destructuring via iteration

julia> A ≈ U * Diagonal(S) * V'
true

julia> Uonly, = svd(A); # Store U only

julia> Uonly == U
true
```

---

```
svd(A, B) -> GeneralizedSVD
```

Compute the generalized SVD of `A` and `B`, returning a `GeneralizedSVD` factorization object `F` such that `[A;B] = [F.U * F.D1; F.V * F.D2] * F.R0 * F.Q'`

  * `U` is a M-by-M orthogonal matrix,
  * `V` is a P-by-P orthogonal matrix,
  * `Q` is a N-by-N orthogonal matrix,
  * `D1` is a M-by-(K+L) diagonal matrix with 1s in the first K entries,
  * `D2` is a P-by-(K+L) matrix whose top right L-by-L block is diagonal,
  * `R0` is a (K+L)-by-N matrix whose rightmost (K+L)-by-(K+L) block is          nonsingular upper block triangular,

`K+L` is the effective numerical rank of the matrix `[A; B]`.

Iterating the decomposition produces the components `U`, `V`, `Q`, `D1`, `D2`, and `R0`.

The generalized SVD is used in applications such as when one wants to compare how much belongs to `A` vs. how much belongs to `B`, as in human vs yeast genome, or signal vs noise, or between clusters vs within clusters. (See Edelman and Wang for discussion: https://arxiv.org/abs/1901.00485)

It decomposes `[A; B]` into `[UC; VS]H`, where `[UC; VS]` is a natural orthogonal basis for the column space of `[A; B]`, and `H = RQ'` is a natural non-orthogonal basis for the rowspace of `[A;B]`, where the top rows are most closely attributed to the `A` matrix, and the bottom to the `B` matrix. The multi-cosine/sine matrices `C` and `S` provide a multi-measure of how much `A` vs how much `B`, and `U` and `V` provide directions in which these are measured.

# Examples

```jldoctest
julia> A = randn(3,2); B=randn(4,2);

julia> F = svd(A, B);

julia> U,V,Q,C,S,R = F;

julia> H = R*Q';

julia> [A; B] ≈ [U*C; V*S]*H
true

julia> [A; B] ≈ [F.U*F.D1; F.V*F.D2]*F.R0*F.Q'
true

julia> Uonly, = svd(A,B);

julia> U == Uonly
true
```


In [43]:
#First we compute the Schur decomposition TAU0=Q'*S*Z' and TAU1=Q'*T*Z'

r = schur(complex(TAU0),complex(TAU1))
tol_=1e-8;
cutoff=1
sel = (abs.(diag(r.T)./diag(r.S)).<cutoff)

ordschur!(r,sel)
S = r.S
T = r.T
Q = r.Q'  # So now Q*TAU0*Z = S and Q*TAU1*Z = T
Z = r.Z


8×8 Array{Complex{Float64},2}:
        0.0+0.0im        0.0+0.0im  …  0.0+0.0im  0.0+0.0im
   0.219913-0.0im  -0.595879+0.0im     0.0+0.0im  0.0+0.0im
  -0.146609+0.0im   0.397253+0.0im     0.0+0.0im  0.0+0.0im
   -0.93815+0.0im   -0.34623+0.0im     0.0+0.0im  0.0+0.0im
   0.219913-0.0im  -0.595879+0.0im     0.0+0.0im  0.0+0.0im
        0.0+0.0im        0.0+0.0im  …  0.0+0.0im  1.0+0.0im
        0.0+0.0im        0.0+0.0im     1.0+0.0im  0.0+0.0im
 -0.0407169-0.0im   0.110327+0.0im     0.0+0.0im  0.0+0.0im

In [44]:
Q1          = Q[1:neq-m,:]

6×8 Array{Complex{Float64},2}:
      -0.0+0.0im       -0.0+0.0im  …       -0.0+0.0im  -0.186563+0.0im
      -0.0+0.0im       -0.0+0.0im          -0.0+0.0im  -0.559596+0.0im
   0.97285-0.0im        0.0-0.0im           0.0-0.0im   0.186885-0.0im
 -0.115906-0.0im   0.865558-0.0im           0.0-0.0im    0.39342-0.0im
 0.0353251-0.0im  0.0883129-0.0im     -0.984329-0.0im  -0.119904-0.0im
  0.197184-0.0im   0.492961-0.0im  …   0.176341-0.0im  -0.669301-0.0im

In [45]:
Q2          = Q[neq-m+1:neq,:]


2×8 Array{Complex{Float64},2}:
 0.0-0.0im  0.0-0.0im  0.0-0.0im  …  0.0-0.0im  0.0-0.0im  0.0-0.0im
 0.0-0.0im  0.0-0.0im  0.0-0.0im     1.0-0.0im  0.0-0.0im  0.0-0.0im

In [46]:
Q2*PI

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [47]:
Q1*PI

6×2 Array{Complex{Float64},2}:
       0.0+0.0im  -0.186563+0.0im
       0.0+0.0im  -0.559596+0.0im
       0.0+0.0im   0.186885+0.0im
       0.0+0.0im    0.39342+0.0im
 -0.984329+0.0im  -0.119904+0.0im
  0.176341+0.0im  -0.669301+0.0im

In [67]:
r_svd       = svd(Q1*PI)

SVD{Complex{Float64},Float64,Array{Complex{Float64},2}}
U factor:
6×2 Array{Complex{Float64},2}:
       0.0+0.0im  -0.186563+0.0im
       0.0+0.0im  -0.559596+0.0im
       0.0+0.0im   0.186885+0.0im
       0.0+0.0im    0.39342+0.0im
  0.984329+0.0im  -0.119904+0.0im
 -0.176341+0.0im  -0.669301+0.0im
singular values:
2-element Array{Float64,1}:
 1.0000000000000002
 1.0000000000000002
Vt factor:
2×2 Array{Complex{Float64},2}:
 -1.0+0.0im  -0.0+0.0im
  0.0+0.0im   1.0+0.0im

In [68]:
r_svd.V.*r_svd.S.*r_svd.U'

LoadError: [91mDimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 2 and 6")[39m

In [70]:
# Calculate the number of unstable eigenvalues
m = neq-sum(sel);

Q1          = Q[1:neq-m,:];
Q2          = Q[neq-m+1:neq,:];
Z1          = Z[:,1:neq-m]';
Z2          = Z[:,neq-m+1:neq]';

r_svd       = svd(Q2*PI);
U1      = r_svd.U
D1      = r_svd.S
V1      = r_svd.V  
r       = rank(Q2*PI);               # number of linearly independent forecast errors


T11         = T[1:neq-m,1:neq-m];
T12         = T[1:neq-m,neq-m+1:neq];
T22         = T[neq-m+1:neq,neq-m+1:neq];

# T1i         = inv(T11);
S11         = S[1:neq-m,1:neq-m];
S12         = S[1:neq-m,neq-m+1:neq];
S22         = S[neq-m+1:neq,neq-m+1:neq];

r_svd       = svd(Q1*PI);
U1      = r_svd.U
D1      = r_svd.S
V1      = r_svd.V  

r1           = rank(Q1*PI);

r_svd       = svd(Q2*PI);
U2      = r_svd.U
D2      = r_svd.S
V2      = r_svd.V  
r2           = rank(Q2*PI);

In [75]:
V1.*D1

2×2 Array{Complex{Float64},2}:
 -1.0-0.0im  0.0-0.0im
 -0.0-0.0im  1.0-0.0im

In [41]:
Sigma         = ((U2[:,1:r2].*(D2[1:r2,1:r2]\V2[:,1:r2]')).*(V1[:,1:r1].*D1[1:r1,1:r1].*U1[:,1:r1]'))'

LoadError: [91mBoundsError: attempt to access 2-element Array{Float64,1} at index [1:2, 1:2][39m

In [12]:
Sigma         = ((U2[:,1:r2].*(D2[1:r2,1:r2]\V2[:,1:r2]')).*(V1[:,1:r1].*D1[1:r1,1:r1].*U1[:,1:r1]'))'

#Transfo     = [ones(neq-m,neq-m) -Sigma]
# MY0         = inv([Transfo*S;zeros(m,neq-m) ones(m,m)])
#MY0         = inv([S11 S12-Sigma*S22;zeros(m,neq-m) ones(m,m)])

LoadError: [91mBoundsError: attempt to access 2-element Array{Float64,1} at index [1:2, 1:2][39m

In [4]:
Sigma         = ((U2[:,1:r2].*(D2[1:r2,1:r2]\V2[:,1:r2]')).*(V1[:,1:r1].*D1[1:r1,1:r1].*U1[:,1:r1])')'


Transfo     = [ones(neq-m,neq-m) -Sigma];
MY0         = inv([Transfo*S;zeros(m,neq-m) ones(m,m)]);
#MY0         = inv([T11 T12-Phi*T22;zeros(m,n-m) ones(m)]);
MY1         = ([Transfo*T;zeros(m,neq)]);
MY          = Z*(MY0*MY1)*Z';

if neta>r2;    
    println("Indeterminacy: adding beliefs")
    ME          = MY0*[Transfo*Q*[PHI PI];zeros(m,size([PHI PI],2))];
    ME          = Z*ME;
    ETA         = (U2[:,1:r2]*(D2[1:r2,1:r2]\V2[:,1:r2]'))'*Q2*[PHI PI];
else
    ME          = MY0*[Transfo*Q*PHI;zeros(m,size(PHI,2))];
    ME          = Z*ME;
    ETA         = (U2[:,1:r2]*(D2[1:r2,1:r2]\V2[:,1:r2]'))'*Q2*PHI;
end

LoadError: [91mDimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 6 and 2")[39m

In [ ]:
#solutions
MY  = real(MY);
ME  = real(ME);
ETA = real(ETA);


In [ ]:
# simulation
# Step 4: Recover the impact function

PIE=PIE-PIN*ETA;

#horizon of responses
nrep    = 20;           
YS      = zeros(3,nrep);
XS      = zeros(4,nrep);
Shock   = 1;
YS[:,1] = ME*Shock;
XS[:,1] = PIE;
for t=2:nrep;
    YS[:,t] = MY*YS[:,t-1];
    XS[:,t] = PIY*YS[:,t-1];
end

#Pkg.add("Plots")
using Plots
pyplot()
#plotly() # Choose the Plotly.jl backend for web interactivity
plot(XS[1,:],linewidth=1,label="output",title="Impulse Response Function to a 1% technological shock")
plot!(XS[3,:],linewidth=1,label="investment")


